<a href="https://colab.research.google.com/github/BernardoAIO/Fastcamp-Agents-Activ2/blob/main/ReActAgentFromScratch_aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs e imports

In [ ]:
!pip install groq

In [ ]:
import os
from groq import Groq
from google.colab import userdata

# Teste Inicial

Uso de um completion básico para testar a funcionalidade da API do Groq e a key que será utilizada

In [ ]:
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Requirements**: Many applications, such as voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications such as language translation, sentiment analysis, and text summarization.
4. **Energy Efficiency**: Fast

# Setup da Classe do Agente

* Inicialização (__init__): Configura o agente com um cliente (para acessar a
API) e opcionalmente uma mensagem de sistema, que pode ser utilizada para fornecer contexto ou instruções iniciais ao modelo.  

* Chamada do Agente (__call__): Permite usar a instância do agente como se fosse uma função. Quando chamada, adiciona uma nova mensagem do usuário ao histórico, solicita a resposta do modelo via o método execute, registra essa resposta no histórico como uma mensagem do assistente e, por fim, retorna a resposta.  

* Execução da Chamada à API (execute): Envia o histórico de mensagens para o modelo de linguagem fazer um completion.

In [ ]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

# Prompt

Define o comportamento do agente, orientando-o a seguir um ciclo ReAct (Thought → Action → PAUSE → Observation) até chegar à resposta final.  

**Estrutura ReAct:**
* Thought: O agente descreve seu raciocínio.
* Action: O agente executa uma ação (por exemplo, realizar um cálculo ou buscar a massa de um planeta).
* PAUSE: Uma indicação para esperar o resultado da ação executada.
* Observation: O agente recebe o resultado da ação e utiliza essa informação para continuar o raciocínio.  

**Ações/Tools Disponíveis:**  
* calculate: para operações matemáticas.  
* get_planet_mass: para recuperar a massa de um planeta.

In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

# Funções/Tools do Agente

**calculate:**  
* Avalia uma string que contém uma operação matemática (por exemplo, "4 * 7 / 3") usando a função eval e retorna o resultado numérico.

**get_planet_mass:**  
* Recebe o nome de um planeta, padroniza o texto para minúsculas e, usando um match-case, retorna a massa do planeta correspondente.  
* Se o planeta não estiver listado, retorna 0.0, indicando que o planeta não foi reconhecido.

In [ ]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [ ]:
# Teste de funcionamento da função que obtém a massa dos planetas
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


# Agente sem loop ("manual")

Abaixo, o agente recebe uma query e irá tentar gerar a resposta, porém, ele não tem acesso direto às tools.  

Desse modo, ele irá realizar um pensamento, "solicitar" o uso de uma ação e nós a executaremos de forma manual, esse processo será necessário até chegar na resposta final.

In [ ]:
# Instaciando o Agente "Neil Tyson"
neil_tyson = Agent(client=client, system=system_prompt)

In [ ]:
# Simulando uma entrada/pergunta/query inicial de usuário
result = neil_tyson("what is the mass of Earth times 5?")
print(result)

Thought: I need to find the mass of Earth


In [ ]:
# Agente é executado novamente e identifica/declara que vai precisar realizar ação
result = neil_tyson()
print(result)

Action: get_planet_mass: Earth
PAUSE


In [ ]:
# Utilizamos a tool de forma "Manual"
result = get_planet_mass("Earth")
print(result)

5.972e+24


In [ ]:
# Passamos o resutado como observation, semelhante ao que aconteceria de forma automatizada
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 5.972e+24'

In [ ]:
# Agente recebe a observação e define os próximos passos para responder a query
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5


In [ ]:
# Agente é executado novamente e identifica/declara que vai precisar realizar ação
result = neil_tyson(next_prompt)
print(result)

Action: calculate: 5.972e24 * 5
PAUSE


In [ ]:
# Utilizamos a tool de forma "Manual"
result = calculate("5.972e24 * 5")
print(result)

2.9860000000000004e+25


In [ ]:
# Passamos o resutado como observation, semelhante ao que aconteceria de forma automatizada
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 2.9860000000000004e+25'

In [ ]:
# A partir de todas as ações e observações o agente foi capaz de gerar uma resposta final
result = neil_tyson(next_prompt)
result

'Answer: The mass of Earth times 5 is 2.9860000000000004e+25.'

In [ ]:
# Content das mensagens que o agente recebeu nas últimas celúlas, incluindo entradas, pensamentos e resultados
for msg in neil_tyson.messages:
    print(msg['content'])

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is

# Agente com loop ("automático")

**Instanciação do Agente:**  
A função cria uma instância de Agent passando o client e o system_prompt previamente definidos.

**Definição das Ferramentas:**  
Uma lista de ferramentas disponíveis (calculate e get_planet_mass) é utilizada para determinar quais ações o agente pode pedir.

**Loop:**  
Envia um prompt para o agente e printa sua resposta. Se a resposta incluir uma ação (indicada por "Action" e "PAUSE"), o código extrai a ação e seu argumento utilizando expressões regulares.

Se a ação for válida (estiver na lista de ferramentas), a função correspondente é executada (por exemplo, calcular uma expressão ou obter a massa de um planeta) e o resultado é enviado como observação ao agente.  
Se a resposta contiver "Answer", o loop é interrompido.

Ou seja, essa função automatiza o processo que foi feito de forma manual anteriormente, em que o agente "pedia" para usar uma ação, nós fazíamos a execução e depois apenas o resultado era passado de volta ao agente.

In [ ]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_planet_mass"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: I need to find the mass of Earth and Saturn, then add them together and multiply by 2.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to get the mass of Saturn.
Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now I have the mass of Earth and Saturn, I need to add them together.
Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: Now I have the sum of the masses, I need to multiply it by 2.
Action: calculate: 5.74272e26 * 2
PAUSE
Observation: 1.148544e+27
Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.
